In [1]:
import json
import pandas as pd
import shapely
import geopandas as gpd
from shapely.geometry import LineString, Point
import pandana
import osmnet

In [2]:
lat_min = 42.904155
long_min = -2.621987
lat_max = 43.403070
long_max = -1.740334

Drive

In [3]:
drive_nodes, drive_edges = osmnet.load.network_from_bbox(
    lat_min=lat_min, lng_min=long_min, lat_max=lat_max,
    lng_max=long_max, network_type='drive', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](42.90415500,-2.62199057,43.15677470,-2.17936200);>;);out;'}"
Downloaded 13,152.7KB from www.overpass-api.de in 11.98 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](43.15445613,-2.62199057,43.40391960,-2.17600228);>;);out;'}"
Downloaded 10,359.7KB from www.o

In [4]:
drive_nodes=drive_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [5]:
# Add lat and long to each edge

drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [6]:
drive_edges['geometry']=drive_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [9]:
# Create geodataframe

drive_edges_gdf=gpd.GeoDataFrame(data=drive_edges, geometry='geometry', crs='epsg:4326')

Ensure all roads have valid speed

In [10]:
# Delete "_link"

drive_edges_gdf['highway']=drive_edges_gdf['highway'].apply(lambda hw: hw.replace('_link', ''))

# Fill null values
# Then, change from "60 km/h" to "60"

drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].fillna(0)
drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].apply(lambda s: int(str(s).split(' ')[0]))

# Group highways with type and save mode for each type

grouped = drive_edges_gdf.loc[drive_edges_gdf['maxspeed']>0].groupby('highway')['maxspeed'].agg(pd.Series.mode)
for t in drive_edges_gdf.highway.unique():
    if t not in grouped.keys():
        grouped[t] = grouped["unclassified"]

# Replace zero values with group mode
drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'maxspeed'] = drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'highway'].apply(lambda x: grouped[x])

In [19]:
# Add m/s speed and time as a function of distance and speed.

drive_edges_gdf['speed_m_s']=0.6*drive_edges_gdf['maxspeed']*1000/3600
drive_edges_gdf['drive_time_s']=drive_edges_gdf['distance']/drive_edges_gdf['speed_m_s']

Create shapefile for GAMA

In [13]:
# drive_edges_gdf.to_file(f"data/drive_shapefile.shp", driver='ESRI Shapefile')

Create pandana network

In [20]:
drive_net_pdna=pandana.network.Network(
    drive_nodes['lon'], drive_nodes['lat'],
    drive_edges_gdf['from'], drive_edges_gdf['to'],
    drive_edges_gdf[['distance','speed_m_s','drive_time_s']], twoway=False)

In [21]:
drive_net_pdna.save_hdf5("../networks/drive_net.h5")

In [18]:
drive_nodes.to_csv("../networks/drive_nodes.csv")

In [19]:
# drive_edges_gdf.to_file('../data/drive_net.geojson', index=False)
# drive_edges_gdf.to_file('../front_end/data/drive_net.geojson', index=False)
# drive_nodes.to_csv('../data/drive_nodes.csv')

Walk and Bike

In [20]:
walk_nodes, walk_edges = osmnet.load.network_from_bbox(
    lat_min=lat_min, lng_min=long_min, lat_max=lat_max,
    lng_max=long_max, network_type='walk', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](42.90415500,-2.62199057,43.15677470,-2.17936200);>;);out;'}"
Downloaded 46,332.1KB from www.overpass-api.de in 1.89 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](43.15445613,-2.62199057,43.40391960,-2.17600228);>;);out;'}"
Downloaded 33,014.9KB from www.overpass-api.de in 1.58 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!

In [21]:
walk_nodes=walk_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [22]:
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [23]:
walk_edges['geometry']=walk_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [24]:
walk_edges_gdf=gpd.GeoDataFrame(data=walk_edges, geometry='geometry', crs='epsg:4326')

In [26]:
walk_nodes.to_csv("../networks/walk_nodes.csv")

Create walk pandana network

In [27]:
walk_net_pdna=pandana.network.Network(
    walk_nodes['lon'], walk_nodes['lat'],
    walk_edges_gdf['from'], walk_edges_gdf['to'],
    walk_edges_gdf[['distance']], twoway=False)

In [28]:
walk_net_pdna.save_hdf5("../networks/walk_net.h5")

In [29]:
# walk_edges_gdf.to_file('../data/walk_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/walk_net.geojson', index=False)
# walk_nodes.to_csv('../data/walk_nodes.csv')

Create bike pandana network

In [30]:
walk_net_pdna.save_hdf5("../networks/bike_net.h5")

In [31]:
# walk_edges_gdf.to_file('../data/bike_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/bike_net.geojson', index=False)
# walk_nodes.to_csv('../data/bike_nodes.csv')